# SINE WAVE MODEL
This script is to develop a model to predict the sine of a value. The model is deployed to a stm32 board (nucleo-l476rg). The model is converted to a tflite model to be run with the tensorflowlite-micro tflm frame work.

## Importing the needed library

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

## Defining necessary libraries

In [ ]:
SAMPLE_SIZE = 1_000
RANDOM_SEED = 25
PI = 3.14159265359

MODEL_FILE_NAME = "SINE_MODEL"
C_HEADER_DIR = "../deployment/"

train_ratio = .7
test_ratio = .2
validation_ratio = .1

## Generating the dataset
The data will contain numbers between 0 and 2pi, the sine function will be defined in rad.
A random uniform error is added to the sine values to give the dataset a bit of variability and see how the model will perform.
The error is centred with a mean of 0 and std 0f 0.05. This is chosen such that 95% of the error is -0.1 and 0.+1.

In [ ]:
np.random.seed(RANDOM_SEED) # for reproduction
x = np.random.uniform(low=0, high=2*PI, size=SAMPLE_SIZE)
y = np.sin(x) + np.random.normal(0, 0.05, size=SAMPLE_SIZE)
x.shape, y.shape

### Plot of the data
1. The distribution of the imput data.
2. The relationship between the output y and x i.e. the sine curve.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16,6))

axes[0].hist(x, bins=20)
axes[0].set_title("Distribution of the datapionts.")
axes[0].set_xlabel("x")
axes[0].set_ylabel("frequency")


axes[1].scatter(x,y, marker=".")
axes[1].set_title("The relationship of the x and sine(x)")
axes[1].set_ylabel("sin(x)")
axes[1].set_xlabel("x")

plt.show()

## Preparing the data for training

In [ ]:
# x_train = x[:SAMPLE_SIZE * train_ratio]
# y_train = y[:SAMPLE_SIZE * train_ratio]

# x_test = x[SAMPLE_SIZE * train_ratio : SAMPLE_SIZE * test_ratio]
# y_test = y[SAMPLE_SIZE * train_ratio : SAMPLE_SIZE * test_ratio]

# x_validate = x[SAMPLE_SIZE * (1 - validation_ratio):]
# y_validate = y[SAMPLE_SIZE * (1 - validation_ratio):]

x_train, x_test, x_validate = np.split(x, [int(train_ratio*SAMPLE_SIZE), int((train_ratio+test_ratio)*SAMPLE_SIZE)])
y_train, y_test, y_validate = np.split(y, [int(train_ratio*SAMPLE_SIZE), int((train_ratio+test_ratio)*SAMPLE_SIZE)])

### Make a plot to show to distribution of the training, testing and validation data.

In [ ]:
plt.scatter(x_train, y_train,marker=".", label="Train")
plt.scatter(x_test, y_test, marker=".", label="Test")
plt.scatter(x_validate, y_validate, marker=".", label="Validate")

plt.title("sin(x) vs x")
plt.xlabel("x")
plt.ylabel("sin(x)")
plt.legend()
plt.show()

## Model Definition
A simple  model will be used to test the framework. A 2 [hidden] layer model with the following:
- Input layer -> 1 neuron
- Hidden layer 1 -> 32 neurons
- Hidden layer 2 -> 16 neurons
- Output layer -> 1 neuron

In [ ]:
sine_model = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation="relu", input_shape=(1,)),
    tf.keras.layers.Dense(16, activation="relu"),
    tf.keras.layers.Dense(1)
])

In [ ]:
sine_model.summary()

In [ ]:
sine_model.compile(optimizer="adam", loss="mse", metrics=["mae"])
history = sine_model.fit(x_train, y_train, batch_size=100, epochs=500, validation_data=(x_validate, y_validate))

## Training Plot

In [ ]:
fig, axes = plt.subplots( 1, 2, figsize=(16,6))

axes[0].plot(history.history["loss"], label="training_loss")
axes[0].plot(history.history["val_loss"], label="validation_loss")
axes[0].set_title("Model loss")
axes[0].set_xlabel("epoch")
axes[0].set_ylabel("loss (mse)")

axes[1].plot(history.history["mae"], label="training_mae")
axes[1].plot(history.history["val_mae"], label="validation_loss")
axes[1].set_title("Model MAE")
axes[1].set_xlabel("epoch")
axes[1].set_ylabel("mae")

## Testing the model

In [ ]:
y_test_predict = sine_model.predict(x_test)
y_test_true = np.sin(x_test)

plt.scatter(x_test, y_test_predict, marker=".", label="True value")
plt.scatter(x_test, y_test_true, marker=".", label="Predicted value")
plt.title("Prediction and True Values")
plt.ylabel("sin(x)")
plt.xlabel("x")
plt.legend()

## Converting the model to tflite

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(sine_model)
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
sine_tflite_model = converter.convert()

## Converting the model to the c format

In [ ]:
# Breaking the byte in several lines to fit better in the c header file
sine_tflite_model_split_line = np.array_split([format(hex_value, '#04x') for hex_value in sine_tflite_model], len(sine_tflite_model)//8)

# Write TfLite model to a C header file
open(f"{C_HEADER_DIR}{MODEL_FILE_NAME}.h", "w").write(
f"""
#ifndef {MODEL_FILE_NAME.upper()}_H
#define {MODEL_FILE_NAME.upper()}_H

const unsigned int {MODEL_FILE_NAME.lower()}_len = {len(sine_tflite_model)};

const unsigned char {MODEL_FILE_NAME.lower()}[{len(sine_tflite_model)}] = {{
    {",\n    ".join([", ".join(line) for line in sine_tflite_model_split_line])}
}};

#endif // {MODEL_FILE_NAME.upper()}_H
"""
)

# Saving the TfLite model
open(f"{MODEL_FILE_NAME}.tflite", "wb").write(sine_tflite_model)

## Testing the converted tflite model

In [ ]:
interpreter = tf.lite.Interpreter(model_path=f"{MODEL_FILE_NAME}.tflite")
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

y_test_tflite_predict = []
for x_ in x_test:
  # Set input tensor
  interpreter.set_tensor(input_details[0]['index'], np.array([[x_]], dtype=np.float32))

  # Run the inference
  interpreter.invoke()

  y_test_tflite_predict.append(interpreter.get_tensor(output_details[0]['index']))

y_test_tflite_predict = np.array(y_test_tflite_predict)


In [ ]:
y_test_predict = sine_model.predict(x_test)
y_test_true = np.sin(x_test)

mae_orginal_model = np.sum(np.abs(y_test_predict - y_test)) / y_test.shape[0]
mae_converted_model = np.sum(np.abs(y_test_tflite_predict - y_test)) / y_test.shape[0]

fig, axes = plt.subplots(1, 2, figsize=(16,6)s)

axes[0].scatter(x_test, y_test_tflite_predict, marker=".", label="True value")
axes[0].scatter(x_test, y_test_true, marker=".", label="Predicted value")
axes[0].set_title("Prediction and True Values")
axes[0].set_ylabel("sin(x)")
axes[0].set_xlabel("x")
axes[0].legend()

axes[1].bar(["orginal model", "converted model"], [mae_orginal_model, mae_converted_model])
axes[1].set_title("The effect of coversion to model accuracy")
axes[1].set_ylabel("mae")
axes[1].set_xlabel("model type")
